
# Object Detection Controlled Car

This notebook demonstrates using object detection from a camera stream to control a remote car. By employing a pre-trained YOLO model for object detection and processing the video stream, specific objects can trigger car movements or actions.
    

In [ ]:

import cv2
import numpy as np
import socket
import json
import time
import requests
    


## Loading Class Names and YOLO Model

We begin by loading the class names from COCO dataset and initializing the YOLO model with the pre-trained weights and configuration.
    

In [ ]:

classNames = []
with open('utils/resources/coco.names', 'r') as f:
    classNames = f.read().splitlines()

model = 'utils/resources/yolov3.weights'
config = 'utils/resources/yolov3.cfg'
net = cv2.dnn.readNet(model, config)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)
    


## Car Control Interface Configuration

Next, we configure the car's control interface by setting the IP and port for communication.
    

In [ ]:

IP = "192.168.4.1"
PORT = 100
    


## Function for Sending Commands to the Car

This function packages and sends control commands to the car via a TCP socket connection.
    

In [ ]:

def send_command(command):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((IP, PORT))
        sock.sendall(json.dumps(command).encode())
        time.sleep(0.5)
    


## Processing the Camera Stream for Object Detection

We process frames from the camera stream to detect objects using the YOLO model. Detected objects can then be used to make decisions on how to control the car's movements.
    

In [ ]:

session = requests.Session()
response = session.get(stream_url, stream=True)

if response.status_code == 200:
    bytes_data = bytes()
    while True:
        chunk = response.raw.read(1024)
        if not chunk:
            break
        bytes_data += chunk
        a = bytes_data.find(b'ÿØ')
        b = bytes_data.find(b'ÿÙ')
        while a != -1 and b != -1:
            jpg = bytes_data[a:b+2]
            bytes_data = bytes_data[b+2:]
            img = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            
            if img is not None:
                blob = cv2.dnn.blobFromImage(img, 1 / 255, (320, 320), (0, 0, 0), crop=False)
                net.setInput(blob)
                layerNames = net.getLayerNames()
                outputNames = [layerNames[i-1] for i in net.getUnconnectedOutLayers()]
                outputs = net.forward(outputNames)
                # Object Detection Logic and Command Sending

                cv2.imshow('Elegoo Smart Car Camera Stream', img)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break

            a = bytes_data.find(b'ÿØ')
            b = bytes_data.find(b'ÿÙ')

cv2.destroyAllWindows()
    


## Student Exercises

1. **Implement Object Detection Logic**: Complete the object detection logic to identify specific objects within the video frames. Use the detected objects to decide on the car's movements.

2. **Dynamic Command Decisions**: Based on the detected objects and their positions or sizes, dynamically adjust the commands sent to the car.

3. **Integrate Multiple Object Detection**: Enhance the control logic to handle multiple objects detection simultaneously and prioritize or combine commands accordingly.

4. **Feedback Mechanism**: Explore ways to provide feedback to the user about the detected objects and the car's response to those detections.
    